In [1]:
import numpy as np
import random
import math
import copy
import pprint as pp

In [2]:
max_age = 15

In [3]:
cell_ = {
    'i': None, 
    'j': None, 
    'elev': None, 
    'hum': 0, 
    'zom': [0] * max_age, 
    'z_1': [0] * max_age, 
    'z_0': [0] * max_age 
}

def initializeCell():
    return copy.deepcopy(cell_)

In [4]:
# Determine the killed humans and update its cell value
def zombiesKillHumans(cell):
    h = cell['hum'] # * 255 ? is it average or what?
    z = sum(cell['zom'])
    killed_h = 10 * z
    remaining_h = h - killed_h if h > killed_h else 0
    cell['hum'] = remaining_h

In [53]:
# Determine the killed zombies and update its cell value
def humanKillZombies(cell):
    h = cell['hum']
    z = sum(cell['zom'])
    killed_z = 10 * h
    # assign randomly to the list the killings
    for k in range(killed_z):
        remaining_ages = [a for a in range(max_age) if cell['zom'][a] > 0]
        index = 0
        if len(remaining_ages) == 0:
            return
        elif len(remaining_ages) > 1:
            index = random.randint(0, len(remaining_ages) - 1)
        age = remaining_ages[index]
        cell['zom'][age] -= 1 
        
"""
cell_test = initializeCell()
cell_test['hum'] = 10
for z in range(len(cell_test['zom'])):
    cell_test['zom'][z] = 10

print(sum(cell_test['zom']))
humanKillZombies(cell_test)
print(sum(cell_test['zom']))
"""

"\ncell_test = initializeCell()\ncell_test['hum'] = 10\nfor z in range(len(cell_test['zom'])):\n    cell_test['zom'][z] = 10\n\nprint(sum(cell_test['zom']))\nhumanKillZombies(cell_test)\nprint(sum(cell_test['zom']))\n"

In [6]:
# "consider cells of 15 × 15 pixels" AND "1 pixel = 1 km 2"
len_cell = 15 #km
len_diag = math.sqrt(pow(len_cell, 2) * 2) #km
dist_lin = len_cell
dist_dia = len_diag

def getGeoSlope(cell, to_cell):
    # (i == to_i or j == to_j) means they are vertical or horizontal
    dist = dist_lin if (cell['i'] == to_cell['i'] or cell['j'] == to_cell['j']) else dist_dia
    diff = to_cell['elev'] - cell['elev']
    return math.degrees(math.atan2(diff, dist))

def getGeoSlopeFactor(cell, to_cell):
    slope = getGeoSlope(cell, to_cell)
    if slope < 0: 
        return 1 # zombies don't have any issue
    elif slope > 10:
        return 0 # zombies don't climb
    else:
        # then, between 0 and 10
        return 1 - (slope / 10) # zombies have certain difficulties

In [44]:
def zombiesMove(cell, neighbor_cells_list):
    """ 
    1) thinking that the zombies will all go even if there are still humans
    2) We're assuming, also, that not all zombies should be moved (ex: slope constraint would encourage some to stay)
    """
    
    # today
    neighbor_humans = []
    for c in range(8): # 8 cells
        neighbor_humans.append(neighbor_cells_list[c]['hum'])
    h_neigh = sum(neighbor_humans)
    elev_c0 = cell['elev'] 
    #print(f'h_neigh: {h_neigh}')
    # tomorrow
    autoDestroyZombies(cell) # prepare zombies list for "tomorrow"
    z_c0 = sum(cell['zom'])
    for c in range(8): # 8 cells
        z = sum(neighbor_cells_list[c]['zom'])
        h = neighbor_cells_list[c]['hum']
        geo_slope_fact = getGeoSlopeFactor(cell, neighbor_cells_list[c]) # (from, to)
        z_new = getZombiesContribution(cell, h_neigh, z_c0, geo_slope_fact)
        #print(f'z_new {z_new}')
        assignZombiesByAge(cell, neighbor_cells_list[c], z_new)
                

In [55]:
neigh = getNeigborCellList(2, 2, m)
cell = m[2][2]
zombiesMove(cell, neigh)
zombiesKillHumans(m[1][1])
humanKillZombies(m[1][1])
pp.pprint(showMatrixVals(m, 'hum'))
pp.pprint(showMatrixVals(m, 'zom'))
z = sum([sum(y) for e in showMatrixVals(m, 'zom') for y in e])
print(z)

h_neigh: 740
[[100, 100, 100, 100, 100],
 [100, 30, 100, 100, 100],
 [100, 100, 100, 100, 100],
 [100, 100, 100, 100, 100],
 [100, 100, 100, 100, 100]]
[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 4, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 

In [45]:
def getZombiesContribution(cell, h_neigh, z_c0, geo_slope_fact):
    z = sum(cell['zom'])
    h = cell['hum']
    #print(f'h cell: {h}, z_c0: {z_c0}, h_neigh: {h_neigh}')
    if h_neigh == 0:
        return 0
    elif h_neigh > 0:
        return math.ceil((h / h_neigh) * z_c0 * geo_slope_fact)
    else:
        print("Error in Zombies contribution: z < 0 ?")

In [10]:
def autoDestroyZombies(cell):
    #cell['zom'][-1] = 0 # last cell equals 0, (autodestroyed zombies)
    cell['zom'].pop() # remove last element
    cell['zom'].insert(0, 0) # insert new
    
def assignZombiesByAge(cell, new_cell, n):
    for i in range(n):
        remaining_ages = [a for a in range(max_age) if cell['zom'][a] > 0]
        index = 0
        if len(remaining_ages) == 0:
            return
        elif len(remaining_ages) > 1:
            index = random.randint(0, len(remaining_ages) - 1)        
        age = remaining_ages[index]
        cell['zom'][age] -= 1
        new_cell['zom'][age] += 1
    #print(cell['zom'])
    #print(new_cell['zom'])

In [11]:
cell_test = initializeCell()
cell_test2 = initializeCell()
for z in range(len(cell_test['zom'])):
    cell_test['zom'][z] = 10
    cell_test2['zom'][z] = 10
print(sum(cell_test['zom']))
print(sum(cell_test2['zom']))
assignZombiesByAge(cell_test, cell_test2, 5)
print(sum(cell_test['zom']))
print(sum(cell_test2['zom']))

150
150
145
155


In [12]:
# If still humans in the place where zombies are, this will move inspite humans in cell?
# Averages, should we calculate humans or average humans?

In [13]:
obj1 = {'h': 1}
obj2 = {'h': 2}

def rename(ob):
    for c in range(len(ob)):
        ob[c]['h'] += 1

rename([obj1, obj2])

print(obj1)
print(obj2)

{'h': 2}
{'h': 3}


In [14]:
# Rize pop 94 800 inhabitants


In [51]:
# TEST
m = [[initializeCell() for i in range(5)] for j in range(5)]
elev = 0
for i in range(5):
    for j in range(5):
        m[i][j]['elev'] =  elev + 1
        m[i][j]['i'] = i
        m[i][j]['j'] = j
        m[i][j]['hum'] = 100
        elev += 1
        
m[2][2]['zom'][0] = 30

In [16]:
def showMatrixVals(m, key):
    new_m = [[0 for i in range(5)] for j in range(5)]
    for i in range(5):
        for j in range(5):
            new_m[i][j] = m[i][j][key]
    return new_m

In [17]:
def getNeigborCellList(i, j, m):
    n = []
    for i_sub in range(3):
        for j_sub in range(3):
            if i_sub == 1 and j_sub == 1: continue 
            candidate_cell = m[i + i_sub - 1][j + j_sub - 1]
            if isCellCandidateForMove(candidate_cell): n.append(candidate_cell)
    return n

In [56]:
def isCellCandidateForMove(c):
    return True # MODIFY TO DETECT BORDERS OR CELLS NOT DESIRED

In [18]:
neigh = getNeigborCellList(2, 2, m)
cell = m[2][2]
zombiesMove(cell, neigh)

h_neigh: 80
h cell: 10
z_new 4
h cell: 10
z_new 4
h cell: 10
z_new 4
h cell: 10
z_new 4
h cell: 10
z_new 4
h cell: 10
z_new 4
h cell: 10
z_new 4
h cell: 10
z_new 4


In [19]:
pp.pprint(showMatrixVals(m, 'hum'))
pp.pprint(showMatrixVals(m, 'zom'))

[[10, 10, 10, 10, 10],
 [10, 10, 10, 10, 10],
 [10, 10, 10, 10, 10],
 [10, 10, 10, 10, 10],
 [10, 10, 10, 10, 10]]
[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],


In [ ]:
def runOneMoreDay(m, is_0_layer):
    # STEP 1 (Zombies move)
    for lat in m:
        for lon in lat:
            now_cell = m[lat][lon]
            neighbor_cells = getNeigborCellList(now_cell)
            zombiesMove(now_cell, neighbor_cells)
    # STEP2 (Zombies Kill)
    for lat in m:
        for lon in lat:
            zombiesKillHumans(m[lat][lon])
    # STEP 3 (Humans kill)
    for lat in m:
        for lon in lat:
            humanKillZombies(m[lat][lon])